In [1]:
# Import qiskit and modules
import matplotlib.pyplot as plt
import numpy as np
import math
import random
import time

from math import pi

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, transpile, assemble
from qiskit import Aer
from qiskit.tools.visualization import circuit_drawer
from qiskit.quantum_info import state_fidelity
from qiskit import BasicAer
from qiskit.circuit.library import TwoLocal

from scipy.optimize import minimize

In [2]:
from AtomLoader import *

In [3]:

backend = BasicAer.get_backend('statevector_simulator')


In [4]:
def training_circuit(parameters, atomic_descriptors):
    
    qc = QuantumCircuit(M)
    qc_descriptor = QuantumCircuit(M)
    
    list_eta = parameters[0:M]
    list_ksi = parameters[M:M + (depth+1)*M*2] ## twolocal have 4M parameters
    
    # for i in range(M):
    for i, descriptor in enumerate(atomic_descriptors):
        qc_descriptor.u(
            descriptor[0],
            descriptor[1],
            0,
            i
            ) #should be replaced to theta(list_eta[i]), phi(list_ksi[i])
        #Young Oh should read this!

    twolocal = TwoLocal(num_qubits=M, reps=depth, rotation_blocks=['ry','rz'], 
                   entanglement_blocks='cx', entanglement='circular', parameter_prefix='ξ', insert_barriers=True)
    twolocal = twolocal.bind_parameters(list_ksi)
    
    qc += qc_descriptor
    qc.barrier()

    qc += twolocal
    qc.barrier()

    #Observable
    qc.z(0)

    qc.barrier()

    qc += twolocal.inverse()
    qc.barrier()

    qc += qc_descriptor.inverse()
    qc.barrier()
    
    return qc

In [10]:
def calculate_loss_function(parameters, random_indexes):

    
    backend = Aer.get_backend('aer_simulator')
    loss = 0
    
    for molecule_index in random_indexes:
        energy = 0
        list_eta = np.array(parameters[0:M])
        
        atom_data = AtomLoader(list_eta, idx=[molecule_index])
        ground_energy_label = atom_data[molecule_index]['ground_energy'][0]/-20000
        descriptors = atom_data[molecule_index]['descriptor']
        n_atoms = len(atom_data[molecule_index]['descriptor'])
        # for i in range(0, N):

        for i, atomic_descriptors in enumerate(descriptors):

            qctl = QuantumRegister(M)
            qc = ClassicalRegister(M)
            circ = QuantumCircuit(qctl, qc)
        
            circ += training_circuit(parameters, atomic_descriptors)
            
            circ.save_statevector()
            t_circ = transpile(circ, backend)
            qobj = assemble(t_circ)
            job = backend.run(qobj)

            result = job.result()
            outputstate = result.get_statevector(circ, decimals=100)
            o = outputstate

            energy += np.real(o[0]) ## <0|GdWdOWG|0> is picking first component of GdWdOWG|0>. I don't think square is necessary
        
        loss += ((energy - ground_energy_label)/n_atoms)**2

    return np.sqrt(loss/len(random_indexes))

In [ ]:
M = 4 #Number of descriptors (features) = Number of qubits
depth = 2 #Number of repetitions of layer
len_param = M*(2*depth+3) #Number of total parameters (eta + ksi)

print("Number of total parameters: ", len_param)

random_indexes = [str(x) for x in (np.random.choice(len(qm9),5 ) + 1)]

print(calculate_loss_function(x0,random_indexes))

class OpObj(object):
    def __init__(self):
        #맨처음 params의 초기값 
        random.seed(81)
        self.x_0 = np.linspace(0, 1, M).tolist() + [float(random.randint(0,3000))/1000 for i in range(0, len_param-M)]
        #minimize 중간에 뽑아져 나오는 값을 담을 변수
        self.f = np.full(shape=(iter,), fill_value=np.NaN)
        #그걸 f 의 몇번째에 넣을지 계산하는 변수 
        self.count = 0
        

start = time.time()
out = minimize(calculate_loss_function, 
    x0, 
    method="COBYLA", 
    options={'maxiter':100},
    args=(random_indexes)
    )
end = time.time()

print(f"Calculation time: {end-start}s\n")

print(out)

#out_f = [out['x'][0:3], out['x'][3:6], out['x'][6:9]]

Number of total parameters:  28
0.2513845546617986


In [15]:
random_indexes

['43324']

In [23]:

np.save('out.npy',np.array(out.x))

array([2.25016042, 1.19627615, 2.4758455 , 3.54709867, 2.95321155,
       2.70278014, 2.32553127, 2.11312249, 2.09122828, 1.07384407,
       1.91246993, 1.86939811, 0.13562813, 0.4481549 , 1.9255319 ,
       1.50425959, 1.85176361, 3.42927938, 2.29592876, 0.12867847,
       2.45367396])

In [42]:
molecule_index = '28970' 
list_eta = out.x[0:M]
atom_data = AtomLoader(list_eta, idx=[molecule_index])
ground_energy_label = atom_data[molecule_index]['ground_energy'][0]/-20000
descriptors = atom_data[molecule_index]['descriptor']
n_atoms = len(atom_data[molecule_index]['descriptor'])
energy = 0
for i, atomic_descriptors in enumerate(descriptors):
    qctl = QuantumRegister(M)
    qc = ClassicalRegister(M)
    circ = QuantumCircuit(qctl, qc)

    circ += training_circuit(out.x, atomic_descriptors)
    
    # circ.save_statevector()
    t_circ = transpile(circ, backend)
    qobj = assemble(t_circ)
    job = backend.run(qobj)

    result = job.result()
    outputstate = result.get_statevector(circ, decimals=100)
    o = outputstate

    energy += np.real(o[0]) ## <0|GdWdOWG|0> is picking first component of GdWdOWG|0>. I don't think square is necessary
print(energy)
print(ground_energy_label)

/home/justids/anaconda3/envs/qiskit/lib/python3.7/site-packages/ipykernel_launcher.py:23: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
/home/justids/anaconda3/envs/qiskit/lib/python3.7/site-packages/ipykernel_launcher.py:26: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
/home/justids/anaconda3/envs/qiskit/lib/python3.7/site-packages/ipykernel_launcher.py:34: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
/home/justids/anaconda3/envs/qiskit/lib/python3.7/site-pac

0.5385008252893783
0.5385087715150687


In [39]:
ground_energy_label*-20000

-11824.799608105232

In [41]:
random_indexes

['28970']